### Sequential API:
A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor.

A Sequential model is not appropriate when:
1. Your model has multiple inputs or multiple outputs
2. Any of your layers has multiple inputs or multiple outputs
3. You need to do layer sharing
4. You want non-linear topology (e.g. a residual connection, a multi-branch model)


In [1]:
import tensorflow as tf
from tensorflow import keras

In [5]:
layers=keras.layers
mnist = keras.datasets.mnist

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape, y_train.shape)
# 60000 images with w*h=28*28 and 60000 labels
print(x_train.min(), x_train.max())
x_train = x_train.reshape(-1, 28*28).astype('float32')/255.0
x_test = x_test.reshape(-1, 28*28).astype('float32')/255.0

"""
we do not need to convert numpy array to tensor. tensorflow do it internally. 
x_train = tf.convert_to_tensor(x_train) 
"""
# x_train = tf.convert_to_tensor(x_train) 

(60000, 28, 28) (60000,)
0 255


'\nwe do not need to convert numpy array to tensor. tensorflow do it internally. \nx_train = tf.convert_to_tensor(x_train) \n'

In [16]:
model = keras.Sequential([
    keras.Input(28*28),
    layers.Dense(512, activation='relu', name='seq'),
    layers.Dense(256, activation='relu'),
    layers.Dense(10) # output layer
])


In [19]:
model2 = keras.Sequential()
model2.add(keras.Input(shape=(784)))
model2.add(layers.Dense(512, activation="relu"))
model2.add(layers.Dense(256, activation="relu", name="my_layer"))
model2.add(layers.Dense(10))

#### Debug Technique:

In [17]:
model= keras.Model(model.inputs,outputs=[model.layers[-2].output])
# model= keras.Model(model.inputs,outputs=[model.get_layer('seq').output])

feature = model.predict(x_train[:20])
print(feature.shape)

1/1 [==============================] - 0s 187ms/step
(20, 256)


In [18]:
model= keras.Model(model.inputs,outputs=[layer.output for layer in model.layers])
features = model.predict(x_train[:20])

for feature in features:
    print(feature.shape)

1/1 [==============================] - 0s 209ms/step
(20, 784)
(20, 512)
(20, 256)


#### Logits:

In [9]:
"""from_logits = True: will add a softmax into the model then it will map the category
"""

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer= keras.optimizers.Adam(learning_rate=.001),
    metrics= 'accuracy'
)

model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=0)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

157/157 - 1s - loss: 0.0890 - accuracy: 0.9760 - 1s/epoch - 9ms/step


[0.08898523449897766, 0.9760000109672546]

In [6]:
def functional_model():
    inputs =  keras.Input(28*28)
    x = layers.Dense(512,activation='relu', name="first_layer")(inputs)
    x = layers.Dense(256, activation='relu', name="second_layer")(x)
    outputs = layers.Dense(10, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    """from_logits = True: will add a softmax into the model then it will map the category
    """
    model.compile(
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        optimizer= keras.optimizers.Adam(learning_rate=.001),
        metrics= 'accuracy'
    )
    return model
model = functional_model()
print(model.summary())

model.fit(x_train, y_train, batch_size=64, epochs=5, verbose=2)
model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 784)]             0         
                                                                 
 first_layer (Dense)         (None, 512)               401920    
                                                                 
 second_layer (Dense)        (None, 256)               131328    
                                                                 
 dense_5 (Dense)             (None, 10)                2570      
                                                                 
Total params: 535,818
Trainable params: 535,818
Non-trainable params: 0
_________________________________________________________________


None
Epoch 1/5
938/938 - 11s - loss: 0.1997 - accuracy: 0.9399 - 11s/epoch - 12ms/step
Epoch 2/5
938/938 - 10s - loss: 0.0772 - accuracy: 0.9764 - 10s/epoch - 10ms/step
Epoch 3/5
938/938 - 11s - loss: 0.0524 - accuracy: 0.9833 - 11s/epoch - 12ms/step
Epoch 4/5
938/938 - 10s - loss: 0.0368 - accuracy: 0.9876 - 10s/epoch - 11ms/step
Epoch 5/5
938/938 - 10s - loss: 0.0287 - accuracy: 0.9911 - 10s/epoch - 11ms/step
157/157 - 1s - loss: 0.0751 - accuracy: 0.9792 - 880ms/epoch - 6ms/step


[0.0750868022441864, 0.979200005531311]